In [1]:
%load_ext autoreload
%autoreload 2
from config import username, password
print(password)
from modules.login_module import logIn, launch_to_homescreen, create_directory
from modules.download_files_module import request_report_process, download_loop_missing, download_process,  move_files_over, unzip_xlsx_file, unzip_files_in_same_dir, move_xlsx_files
from modules.unit_testing import TestFileProcessing
from modules.data_transformation import *
from modules.post_download_change import *


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchWindowException
from selenium.webdriver.common.action_chains import ActionChains
import os
import pandas as pd
import logging
import time
from datetime import datetime
today_date = datetime.now()
formatted_month_day = today_date.strftime("%m_%d")
pd.set_option('display.max_columns', None)

logging.basicConfig(filename='ELPAC_SBAC_results.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S',force=True)
logging.info('\n\n-------------ELPAC_SBAC_results new instance log')

#create file_download dir, and establish download directory
create_directory('file_downloads')
create_directory(f'file_downloads\\sbac\\{formatted_month_day}')
create_directory(f'file_downloads\\elpac\\{formatted_month_day}')

# Set up Chrome options
download_directory = os.getcwd() + f'\\file_downloads\\elpac\\{formatted_month_day}'
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_directory,
         'profile.default_content_setting_values.automatic_downloads': 1,
         'profile.content_settings.exceptions.automatic_downloads.*.setting': 1}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)

logIn(username, password, driver)
launch_to_homescreen(driver)

coord_list = ['LEA CAASPP Coordinator at Alain Leroy Locke College Preparatory Academy',
 'LEA CAASPP Coordinator at Animo City of Champions Charter High',
 'LEA CAASPP Coordinator at Animo Compton Charter',
 'LEA CAASPP Coordinator at Animo Ellen Ochoa Charter Middle',
 'LEA CAASPP Coordinator at Animo Florence-Firestone Charter Middle',
 'LEA CAASPP Coordinator at Animo Inglewood Charter High',
 'LEA CAASPP Coordinator at Animo Jackie Robinson High',
 'LEA CAASPP Coordinator at Animo James B. Taylor Charter Middle',
 'LEA CAASPP Coordinator at Animo Jefferson Charter Middle',
 'LEA CAASPP Coordinator at Animo Leadership High',
 'LEA CAASPP Coordinator at Animo Legacy Charter Middle',
 'LEA CAASPP Coordinator at Animo Mae Jemison Charter Middle',
 'LEA CAASPP Coordinator at Animo Pat Brown',
 'LEA CAASPP Coordinator at Animo Ralph Bunche Charter High',
 'LEA CAASPP Coordinator at Animo South Los Angeles Charter',
 'LEA CAASPP Coordinator at Animo Venice Charter High',
 'LEA CAASPP Coordinator at Animo Watts College Preparatory Academy',
 'LEA CAASPP Coordinator at Oscar De La Hoya Animo Charter High',
 'LEA ELPAC Coordinator at Alain Leroy Locke College Preparatory Academy',
 'LEA ELPAC Coordinator at Animo City of Champions Charter High',
 'LEA ELPAC Coordinator at Animo Compton Charter',
 'LEA ELPAC Coordinator at Animo Ellen Ochoa Charter Middle',
 'LEA ELPAC Coordinator at Animo Florence-Firestone Charter Middle',
 'LEA ELPAC Coordinator at Animo Inglewood Charter High',
 'LEA ELPAC Coordinator at Animo Jackie Robinson High',
 'LEA ELPAC Coordinator at Animo James B. Taylor Charter Middle',
 'LEA ELPAC Coordinator at Animo Jefferson Charter Middle',
 'LEA ELPAC Coordinator at Animo Leadership High',
 'LEA ELPAC Coordinator at Animo Legacy Charter Middle',
 'LEA ELPAC Coordinator at Animo Mae Jemison Charter Middle',
 'LEA ELPAC Coordinator at Animo Pat Brown',
 'LEA ELPAC Coordinator at Animo Ralph Bunche Charter High',
 'LEA ELPAC Coordinator at Animo South Los Angeles Charter',
 'LEA ELPAC Coordinator at Animo Venice Charter High',
 'LEA ELPAC Coordinator at Animo Watts College Preparatory Academy',
 'LEA ELPAC Coordinator at Oscar De La Hoya Animo Charter High']

elpac_coordinators = [coord for coord in coord_list if 'ELPAC' in coord]
caaspp_coordinators = [coord for coord in coord_list if 'CAASPP' in coord]

#This exists when passing names into the requested reports, as a subset. Change list into a set to only retain unique schools
school_report_names = [entry.split(' at ', 1)[1] for entry in elpac_coordinators]
school_report_names = list(set(school_report_names))

# ---------------------------------------SBAC Files Request and Download

# Call the function, school report names variable is called for just school name
#Equivalent of Student Score Data File
request_report_process(driver, 'SBAC', 'CAASPP_Student_Score_Data_Extract_Report', caaspp_coordinators)
# download_process(school_report_names, '2024 CAASPP Student Score Data File By Enrolled LEA', driver) 

# #This is here three times to see if anything got skipped the first time. Initial dir is set at ELPAC only to move the files over to SBAC dir
# #Will run 5 times

# time.sleep(10) #implemented to give time for files to download
# download_loop_missing(f'elpac\\{formatted_month_day}', '2024 CAASPP Student Score Data File By Enrolled LEA', driver)

# #This moves the files from ELPAC  timestamp dir to SBAC timestamp dir. 
# #This is because the download dir cannot be changed in Selenium
# move_files_over()

# # --------------------------------------------ELPAC Files Request and Download

# driver.switch_to.default_content()
# request_report_process(driver, 'ELPAC', 'Student_Results_Report_Student_Score_Data_Extract', elpac_coordinators)
# download_process(school_report_names, '2024 Summative ELPAC and Summative Alternate ELPAC Student Score Data File By Enrolled LEA', driver) 

# time.sleep(10) #implemented to give time for files to download
# #This is here three times to see if anything got skipped the first time. 
# #Dir remains ELPAC for constant download directory
# download_loop_missing(f'elpac\\{formatted_month_day}', '2024 Summative ELPAC and Summative Alternate ELPAC Student Score Data File By Enrolled LEA', driver)

# #Close out driver window once done
# driver.close()

# #Takes 14 mins to run up to this point
# # -----------------------------------------Unzip the Files and Move them to the P-Drive in this location 'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing'
# unzip_files_in_same_dir('elpac')
# unzip_files_in_same_dir('sbac')

# #Keeps raw zip files in the same dir. Only moves over xlsx files
# try:
#     move_xlsx_files('sbac')
#     logging.info('Moved SBAC XLSX files to p-drive')
# except:
#     logging.info('Unable to move SBAC XLSX files to the p-drive, must be connected to the VPN')
# try:
#     move_xlsx_files('elpac')
#     logging.info('Moved ELPAC XLSX files to p-drive')
# except:
#     logging.info('Unable to move ELPAC XLSX files to the p-drive, must be connected to the VPN')




# #Checks these dirs, for all files being there
# # P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\sbac_01_16
# # P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\elpac_01_16
# test_instance = TestFileProcessing()
# test_instance.test_file_processing('sbac')
# test_instance.test_file_processing('elpac')

# #Takes roughly 25 mins
# #Must be connected to the p-drive

# # ---------------------------------STACKING & SENDING FILES----------------------------------

# directory_path = r'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing'
# formatted_month_day_year = today_date.strftime("%m_%d_%y")


# directory_path_sbac = os.path.join(directory_path, f'sbac_{formatted_month_day_year}')
# sbac = stack_files(directory_path_sbac)
# sbac['CALPADSSchoolCode'] = sbac['CALPADSSchoolCode'].astype(str).str[7:]


# directory_path_elpac = os.path.join(directory_path, f'elpac_{formatted_month_day_year}')
# elpac = stack_files(directory_path_elpac)
# elpac['CALPADSSchoolCode'] = elpac['CALPADSSchoolCode'].astype(str).str[7:]

# sbac = filter_on_full_cds_code(sbac, 'CALPADSSchoolCode')
# elpac = filter_on_full_cds_code(elpac, 'CALPADSSchoolCode')


# # -------------------------------------------get_elpac_import------------------

# def get_elpac_import():

#        e_original = get_elpac_cols(elpac, 'ELPAC')
#        e_scale_score = get_SS_frame(e_original)
#        e_pl_score = get_pl_frame(e_original)

#        #The merge is occurs on testname, and SSID together. THis keeps rows unique
#        e = pd.merge(e_pl_score, e_scale_score, left_on=['SSID', 'testname'], right_on = ['SSID', 'testname'], suffixes= ['', '_SS'], how='left')
#        cols = list(e_pl_score.columns)
#        cols.append('ScaleScore')

#        #re-arrange order
#        col_order = ['Abbreviation', 'SchoolID', 'MasterSchoolID', 'StudentNumber',
#               'StudentID', 'SSID', 'TestGrade', 'ELStatus', 'TestDate', 'DisplayDate',
#               'TestType', 'TestPeriod', 'TestScoreType',  'testname',
#               'ScaleScore', 'PLScore']

#        e = e[col_order]

#        pl_decode = {4.0: 'WelDev', 
#         3.0: 'WelDev', 
#         2.0: 'Som-ModDev',
#         1.0: 'MinDev', 
#         '': 'No Score', 
#         'NS': 'No Score'}

#        e['ProficiencyLevelCode'] = e['PLScore'].map(pl_decode)

#        return(e)

# elpac = get_elpac_import()
# directory_path_elpac = os.path.join(directory_path, f'ELPAC_STACKED_{formatted_month_day_year}.csv')
# elpac.to_csv(directory_path_elpac, index=False)


# #Differing decoding method. Refer to message with Abi
# # ss_decode = {4.0: 'WelDev', 
# #                 3.0: 'ModDev', 
# #                 2.0: 'SomDev',
# #                 1.0: 'MinDev', 
# #                 '': 'No Score', 
# #                 'NS': 'No Score'}

# # -------------------------------Get SBAC import-------------------------

# #Missing PLScore Column and ProficiencyLevelCode Mapping
# sbac_final = get_sbac_cols(sbac, 'SBAC')
# directory_path_sbac = os.path.join(directory_path, f'SBAC_STACKED_{formatted_month_day_year}.csv')
# sbac_final.to_csv(directory_path_sbac, index=False)

# # PL Score 1	STNM
# # PL Score 2	STNL
# # PL Score 3	STMT
# # PL Score 4	STEX

# # ------------------------------Get CAST import-------------------------

# cast = get_cast_cols(sbac, 'CAST')
# directory_path_cast = os.path.join(directory_path, f'CAST_STACKED_{formatted_month_day_year}.csv')
# cast.to_csv(directory_path_cast, index=False)

# # PL Score 1	BLST
# # PL Score 2	ANST
# # PL Score 3	ABST


# #Confirm these are 2024 files

efW5J$kq$Z89AG3
Directory 'file_downloads' already exists.
Directory 'file_downloads\sbac\04_24' created successfully.
Directory 'file_downloads\elpac\04_24' created successfully.


C:\Users\samuel.taylor\AppData\Local\Temp\2\ipykernel_8416\1912884933.py:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)


LEA CAASPP Coordinator at Animo City of Champions Charter High
LEA CAASPP Coordinator at Animo Compton Charter
LEA CAASPP Coordinator at Animo Ellen Ochoa Charter Middle
LEA CAASPP Coordinator at Animo Florence-Firestone Charter Middle
LEA CAASPP Coordinator at Animo Inglewood Charter High
LEA CAASPP Coordinator at Animo Jackie Robinson High
LEA CAASPP Coordinator at Animo James B. Taylor Charter Middle
LEA CAASPP Coordinator at Animo Jefferson Charter Middle
LEA CAASPP Coordinator at Animo Leadership High
LEA CAASPP Coordinator at Animo Legacy Charter Middle
LEA CAASPP Coordinator at Animo Mae Jemison Charter Middle
LEA CAASPP Coordinator at Animo Pat Brown
LEA CAASPP Coordinator at Animo Ralph Bunche Charter High
LEA CAASPP Coordinator at Animo South Los Angeles Charter
LEA CAASPP Coordinator at Animo Venice Charter High
LEA CAASPP Coordinator at Animo Watts College Preparatory Academy
LEA CAASPP Coordinator at Oscar De La Hoya Animo Charter High


In [5]:
pd.read_csv(r'C:\Users\samuel.taylor\Desktop\Python_Scripts\Manual_Triggers\ELPAC_SBAC_Results_Selenium\file_downloads\elpac\04_24\01239920000000\01239920000000_CAASPP_Student_Score_Data_File_EnrolledStudentScoreData_2023_DLMTR.CSV', encoding='latin-1', delimiter='^')

,RecordType,SSID,StudentLastName,StudentFirstName,StudentMiddleName,DateofBirth,Gender,Blank1,Blank2,CALPADSGrade,GradeAssessed,CALPADSDistrictCode,CALPADSDistrictName,CALPADSSchoolCode,CALPADSSchoolName,CALPADSCharterCode,CALPADSCharterSchoolIndicator,SPEDAcctDist,Section504Status,PrimaryDisabilityType,PrimaryDisabilityforTesting,CALPADSIDEAIndicator,IDEAIndicatorforTesting,MigrantStatus,ELStatus,ELEntryDate,RFEPDate,FirstEntryDateInUSSchool,EnrollmentEffectiveDate,ELAS,CEDSLanguageCode,CALPADSPrimaryLanguage,MilitaryStatus,FosterStatus,HomelessStatus,EconomicDisadvantageStatus,EconomicDisadvantageTesting,CALPADSNPSSchoolFlag,HispanicorLatino,AmericanIndianorAlaskaNative,Asian,HawaiianOrOtherPacificIslander,Filipino,BlackorAfricanAmerican,White,TwoorMoreRaces,ReportingEthnicity,ParentEducationLevel,Blank3,OpportunityID1,OpportunityTestingStatus1,OpportunityID2,OpportunityTestingStatus2,OpportunityID3,OpportunityTestingStatus3,OpportunityID4,OpportunityTestingStatus4,TestRegistrationID,TestedDistrictName1,TestedDistrictCode1,TestedSchoolName1,TestedSchoolCode1,TestedCharterSchoolIndicator1,TestedCharterCode1,TestedSchoolNPSFlag1,PaperTestCompletionDate,TestedDistrictName2,TestedDistrictCode2,TestedSchoolName2,TestedSchoolCode2,TestedCharterSchoolIndicator2,TestedCharterCode2,TestedSchoolNPSFlag2,TestedDistrictName3,TestedDistrictCode3,TestedSchoolName3,TestedSchoolCode3,TestedCharterSchoolIndicator3,TestedCharterCode3,TestedSchoolNPSFlag3,TestedDistrictName4,TestedDistrictCode4,TestedSchoolName4,TestedSchoolCode4,TestedCharterSchoolIndicator4,TestedCharterCode4,TestedSchoolNPSFlag4,TestStartDate1,TestCompletedDate1,TestStartDate2,TestCompletedDate2,TestStartDate3,TestCompletedDate3,TestStartDate4,TestCompletedDate4,FinalTestedDistrictName,FinalTestedDistrictCode,FinalTestedSchoolName,FinalTestedSchoolCode,FinalTestedCharterSchoolIndicator,FinalTestedCharterCode,FinalTestedSchoolNPSFlag,FinalTestCompletedDate,SchoolStartDateTestWindow1,SchoolEndDateTestWindow1,SchoolStartDateTestWindow2,SchoolEndDateTestWindow2,SchoolStartDateTestWindow3,SchoolEndDateTestWindow3,SchoolStartDateTestWindow4,SchoolEndDateTestWindow4,StudentExitCode,StudentExitWithdrawalDate,StudentRemovedCALPADSFileDate,ELASCorrectionCode,CASTCurrentYearFlag,CASTParticipatedHighSchoolGrade,CASTParticipatedNPSflag,CASTParticipatedDistrictofAccountability,CASTLastScienceClassFlag,ConditionCode,Attemptedness,ScoreStatus,UnlistedResourcesConstructChange,TestMode,IncludeIndicator,RemoteTester1,RemoteTester2,RemoteTester3,RemoteTester4,SSREligible,ScoreAvailableDate,LexileorQuantileMeasure,GrowthScore,Blank4,RawScore1,RawScore2,RawScore3,RawScore4,Blank5,Blank6,Blank7,Blank8,Blank9,Blank10,Blank11,Blank12,ScaleScore,StandardErrorMeasurement,SmarterScaleScoresErrorBandsMin,SmarterScaleScoresErrorBandsMax,AchievementLevels,Domain1Level,Domain2Level,Domain3Level,Genre,WERPOR,WERDEVEEL,WERCOV,WERPORConditionCode,WERDEVEELConditionCode,WERCOVConditionCode,EAP,ItemsAttempted1,ItemsAttempted2,ItemsAttempted3,ItemsAttempted4,AccommodationsIndicator,DesignatedSupportIndicator,EAAmericanSignLanguage1,EAAmericanSignLanguage2,EAAudioTransScript1,EAAudioTransScript2,EABraille1,EABraille2,EAClosedCaptioning1,EAClosedCaptioning2,EASpeeachtoText1,EASpeeachtoText2,EATexttoSpeech1,EATexttoSpeech2,NEA100NumberTable1,NEA100NumberTable2,NEAAbacus1,NEAAbacus2,NEAAbacus3,NEAAbacus4,NEASupportsforAltAssessments1,NEASupportsforAltAssessments2,NEASupportsforAltAssessments3,NEASupportsforAltAssessments4,NEAAltResponseOptions1,NEAAltResponseOptions2,NEAAltResponseOptions3,NEAAltResponseOptions4,NEABraillePaper,NEACalculator1,NEACalculator2,NEALargePrintSpecialPaper,NEAMultiplicationTable1,NEAMultiplicationTable2,NEAPrintonDemand1,NEAPrintonDemand2,NEAPrintonDemand3,NEAPrintonDemand4,NEAReadAloudPassages1,NEAScribe1,NEAScribe2,NEASpeechtoText1,NEASpeechtoText2,NEAUnlistedResources1,NEAUnlistedResources2,NEAUnlistedResources3,NEAUnlistedResources4,NEAWordPrediction1,NEAWordPrediction2,EDSColorContra

In [ ]:
def request_report(driver, test_type, actual_test):

    # Wait for the button to be clickable based on text
    LEA_Reports = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, f'//button[text()="LEA Reports"]'))
    )

    try:
        # Click the button
        LEA_Reports.click()
        logging.info('LEA Reports clicked')
    except:
        logging.info('LEA reports not clicked')

    iframe = WebDriverWait(driver, 30).until(
    EC.element_to_be_clickable((By.ID, 'theFrame'))
    )
    #must switch to iframe, HTML page built on encapsulation. 
    try:
        driver.switch_to.frame(iframe)
        logging.info('Switched to iframe')
    except:
        logging.info('Unable to swtich to iframe')

    if test_type == 'SBAC':
        # 'CAASPP_Student_Score_Data_Extract_Report'
        caaspp_student_score_data_file = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, f'//option[@value="{actual_test}"]'))
        )
        try:
            caaspp_student_score_data_file.click()
            logging.info('CAASPP Student Score Data Extract Report Clicked')
        except:
            logging.info('CAASPP Student Score Data Extract Report NOT Clicked')

        dropdown_2 = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="caasppschoolYear"]'))
        )
        try:
            dropdown_2.click()
            logging.info('Dropdown Administration Year Selected')
        except:
            logging.info('Unable to select administration year')
   
    elif test_type == 'ELPAC':
        
        elpac_file = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, f'//option[@value="{actual_test}"]'))
        )
        try:
            elpac_file.click()
            logging.info('ELPAC Data File Clicked')
        except:
            logging.info('ELPAC Data File Not Clicked')

        dropdown_2 = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="schoolYear"]'))
        )
        try:
            dropdown_2.click()
            logging.info('Dropdown Administration Year Selected')
        except:
            logging.info('Unable to select administration year')


    else:
        print('Wrong test type')


    try:
        #select SY from second drop down
        SY = '2024'
        schoolyear = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, f'//option[@value="{SY}"]'))
        )
        try:
            schoolyear.click()
            logging.info(f'SY {SY} Selected')
        except:
            logging.info(f'SY {SY} not selected')

        dropdown = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'caasppldrleaidInput'))
        )
        try:
            dropdown.click()
            logging.info('Dropdown clicked')
        except:
            logging.info('Dropdown not clicked')
        
        # Locate and click on the hidden input element, does not need to be done on the first try
        organization = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "cnum1"))
        )
        organization.click()
        logging.info('Organization selected')
    except:
        logging.info('Organization was not selected')

    
    enrolled = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//option[@value="Enrolled"]'))
    )
    try:
        enrolled.click()
        logging.info('Enrolled selected')
    except:
        logging.info('Enrolled not selected')


def change_login_role(school_coord_text, driver):

    # Find the toms-header-container element
    header_container = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'toms-header-container'))
    )

    # Find the role_dropdown element
    role_dropdown = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'selectedRoleOrg'))
    )

    # Scroll into view
    ActionChains(driver).move_to_element(header_container).perform()

    # Now, try clicking on role_dropdown again
    role_dropdown.click()

    #This is where the coordinator role is changed
    option_element = WebDriverWait(driver, 30).until(
    EC.element_to_be_clickable((By.XPATH, f'//option[@class="myTOMS_roleselect_option" and text()="{school_coord_text}"]'))
    )
    option_element.click()



def request_report_process(driver, test_type, actual_test, schools_list):
    for idx, school_coord in enumerate(schools_list):
        
        if idx == 0 and test_type == 'SBAC':
            pass  #LCK is already selected as the initial for SBAC
        else:
            # Change login role for each school_coord
            change_login_role(school_coord, driver)
            logging.info(f'Switched to {school_coord}')
            print(school_coord)
        
        #See if this needs to be here
        reports = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="menu_Reports"]')))
        reports.click()
        
        # Call the function to request a report
        request_report(driver, test_type, actual_test)


In [16]:
directory_path = r'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing'

directory_path_sbac = os.path.join(directory_path, f'sbac_{formatted_month_day}')
sbac = stack_files(directory_path_sbac)
sbac['CALPADSSchoolCode'] = sbac['CALPADSSchoolCode'].astype(str).str[7:]


directory_path_elpac = os.path.join(directory_path, f'elpac_{formatted_month_day}')
elpac = stack_files(directory_path_elpac)
elpac['CALPADSSchoolCode'] = elpac['CALPADSSchoolCode'].astype(str).str[7:]

sbac = filter_on_full_cds_code(sbac, 'CALPADSSchoolCode')
elpac = filter_on_full_cds_code(elpac, 'CALPADSSchoolCode')


# -------------------------------------------get_elpac_import------------------

def get_elpac_import():

       e_original = get_elpac_cols(elpac, 'ELPAC')
       e_scale_score = get_SS_frame(e_original)
       e_pl_score = get_pl_frame(e_original)

       #The merge is occurs on testname, and SSID together. THis keeps rows unique
       e = pd.merge(e_pl_score, e_scale_score, left_on=['SSID', 'testname'], right_on = ['SSID', 'testname'], suffixes= ['', '_SS'], how='left')
       cols = list(e_pl_score.columns)
       cols.append('ScaleScore')

       #re-arrange order
       col_order = ['Abbreviation', 'SchoolID', 'MasterSchoolID', 'StudentNumber',
              'StudentID', 'SSID', 'TestGrade', 'ELStatus', 'TestDate', 'DisplayDate',
              'TestType', 'TestPeriod', 'TestScoreType',  'testname',
              'ScaleScore', 'PLScore']

       e = e[col_order]

       pl_decode = {4.0: 'WelDev', 
        3.0: 'WelDev', 
        2.0: 'Som-ModDev',
        1.0: 'MinDev', 
        '': 'No Score', 
        'NS': 'No Score'}

       e['ProficiencyLevelCode'] = e['PLScore'].map(pl_decode)

       return(e)

elpac = get_elpac_import()
directory_path_elpac = os.path.join(directory_path, f'ELPAC_STACKED_{formatted_month_day}.csv')
elpac.to_csv(directory_path_elpac, index=False)


#Differing decoding method. Refer to message with Abi
# ss_decode = {4.0: 'WelDev', 
#                 3.0: 'ModDev', 
#                 2.0: 'SomDev',
#                 1.0: 'MinDev', 
#                 '': 'No Score', 
#                 'NS': 'No Score'}

# -------------------------------Get SBAC import-------------------------

#Missing PLScore Column and ProficiencyLevelCode Mapping
sbac_final = get_sbac_cols(sbac, 'SBAC')
directory_path_sbac = os.path.join(directory_path, f'SBAC_STACKED_{formatted_month_day}.csv')
sbac_final.to_csv(directory_path_sbac, index=False)

# PL Score 1	STNM
# PL Score 2	STNL
# PL Score 3	STMT
# PL Score 4	STEX

# ------------------------------Get CAST import-------------------------

cast = get_cast_cols(sbac, 'CAST')
directory_path_cast = os.path.join(directory_path, f'CAST_STACKED_{formatted_month_day}.csv')
cast.to_csv(directory_path_cast, index=False)

# PL Score 1	BLST
# PL Score 2	ANST
# PL Score 3	ABST


#Do these get send

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[1 2]


c:\Users\samuel.taylor\Desktop\Python_Scripts\Manual_Triggers\ELPAC_SBAC_Results_Selenium\modules\post_download_change.py:81: FutureWarning: The default value of regex will change from True to False in a future version.
  df['TestName'] = df['TestName'].str.replace(r'(ELA|Math)-', r'\1')


[6]


# Notes 

In [ ]:
#send this file to Abi. 
#original name mapping to new columns

#Abbreviation - map on CALPADSDIstrictName
#schoolid blank
#MasterSchoolID blank
#StudentNumber - LocalStudentId
#StudentID blank
#SSID - SSID
#TestGrade - GradeAssessed
#ELStatus Blank
#TestDate - End of the month for the month uploaded
#DisplayDate blank
#testtype - 21 (General ELPAC) 23(ALT ELPAC)
#testperiod blank
#testscoretype blank
#testname ListeningPL	SpeakingPL	ReadingPL	WritingPL 
#scalescore and plscore melt down
#proficiencylevelcode is hard mapping

#How often do you need this done?
#People dont start asking until end of the testing window, but want a monthly pull. As we lose enrollments of students, we lose access to their scores
#End of the month from Feb through May. Last day of Feb.

#Prioritize ELPAC pulldowns are Feb-May. 
#SBAC is April - June (mid) (End of April for SBAC/CAST)

#CAST has recordtypes that are taken apart within the SBAC file. 

#What table do they send it to?
# ELPAC [TestScores].[dbo].[vw_rpt_ELPACScores]
# CAST  [TestScores].[dbo].[CASTScores]
# SBAC: ELA/Math [TestScores].[dbo].[vw_rpt_SBACScores]